# Training program on AI/ML

# Demo for representation learning

### Linguistic Analogy Evaluation Using FAIR's FastText Embeddings Trained on Wikipedia and News Corpora [Link](https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 76.3 MB/s eta 0:00:00


In [ ]:
!pip install nltk
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.8 MB/s eta 0:00:00


In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
import gensim, logging, os
from gensim import corpora
from collections import defaultdict
import numpy as np
from gensim.models import Word2Vec
import time

In [ ]:
wv_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Datasets/wiki-news-300d-1M.vec', binary=False)

## Word similarities

In [ ]:
sims = wv_model.most_similar('intelligence', topn=10)
for word, score in sims:
    print(f"{word}: {score:.4f}")

## Analogy

In [ ]:
result = wv_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)
print(result) # Expected: [('queen', ...)]

## Dataset loading and preprocessing

In [ ]:
f=open('/content/drive/MyDrive/Datasets/Demo-notebook/sentiment.txt','r')
sentences=f.readlines()
f.close()

In [ ]:
import re

stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

def text_cleaner(text):
    text=remove_link(text.lower())
    long_words=[]
    for i in text.split():
        if i not in stopwords:
            long_words.append(i)
    return long_words

def remove_link(text):
    regex = r'https?://[^\s<>)"‘’]+'
    match = re.sub(regex,' ', text)
    regex = r'https?:|urls?|[/\:,-."\'?!;…]+'
    tweet = re.sub(regex,' ', match)
    tweet = re.sub("[^a-zA-Z_]", " ", tweet)
    tweet = re.sub("[ ]+", " ", tweet)
    return tweet


In [ ]:
vocabulary=[]
X=[]
y=[]
for line in sentences:
	texts=line.strip().lower().split('\t')
	tokens=text_cleaner(texts[0])
	y.append(texts[1])
	X.append(tokens)
	for word in tokens:
		vocabulary.append(word)

## Conversion of text to vector space

In [ ]:
def t2v(tokens,vocabulary):
	vect=[]
	for feature in vocabulary:
		if feature in tokens:
			vect.append(1)
		else:
			vect.append(0)
	return vect

In [ ]:
vector=[]
for tokens in X:
	vect=t2v(tokens,vocabulary)
	vector.append(vect)

In [ ]:
import numpy as np
vector=np.asarray(vector)
y=np.asarray(y)
print("Number of samples,Number of features")
vector.shape


# Feature selection

### Mutual Information based feature selection

In [ ]:
from sklearn.feature_selection import mutual_info_classif
X_new = mutual_info_classif(vector,y)
X_new.shape

In [ ]:
N=len(vocabulary)
avg=0
for mi in X_new:
    avg+=mi
avg=avg/N
feat_selection=[]
for i,mi in enumerate(X_new):
    if mi>avg:
        feat_selection.append(vocabulary[i])
len(feat_selection)

In [ ]:
feat_selection

In [ ]:
vector_feat=[]
for tokens in X:
	vect=t2v(tokens,feat_selection)
	vector_feat.append(vect)

In [ ]:
import numpy as np
vector_feat=np.asarray(vector_feat)
vector_feat.shape

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,feat_selection)
tokens

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,feat_selection)
tokens

# Training Classifier

In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
print("Number of samples, Number of features")
vector_feat.shape

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(vector_feat,y)

In [ ]:
res = model.predict([vect_pos])
res

In [ ]:
res = model.predict([vect_neg])
res

In [ ]:
print("Number of samples, Number of features")
vector.shape

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(vector, y)
t_model = SelectFromModel(lsvc, prefit=True)
X_new = t_model.transform(vector)
X_new.shape

In [ ]:
X_new[1]

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = t_model.transform(np.asarray([vect_pos]))
vect_pos

In [ ]:
res = model.predict(vect_pos)
res

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = t_model.transform(np.asarray([vect_neg]))
vect_neg

In [ ]:
res = model.predict(vect_neg)
res

# Dimensional Reduction

### PCA based dimensional reduction

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
X_new=pca.fit_transform(vector)
X_new.shape

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = pca.transform(np.asarray([vect_pos]))
vect_pos.shape

In [ ]:
res = model.predict(vect_pos)
res

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = pca.transform(np.asarray([vect_neg]))
vect_neg.shape

In [ ]:
res = model.predict(vect_neg)
res

### SVD based dimensional reduction

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
X_new=svd.fit_transform(vector)
X_new.shape

In [ ]:
svml=svm.LinearSVC()
model = svml.fit(X_new,y)

In [ ]:
test_ip="I love watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_pos=t2v(tokens,vocabulary)
vect_pos = svd.transform(np.asarray([vect_pos]))
vect_pos.shape

In [ ]:
test_ip="I hate watching netflix original movies"
tokens=text_cleaner(test_ip)
vect_neg=t2v(tokens,vocabulary)
vect_neg = svd.transform(np.asarray([vect_neg]))
vect_neg.shape

In [ ]:
res = model.predict(vect_pos)
res

In [ ]:
res = model.predict(vect_neg)
res

# Word2Vec

In [ ]:
import gensim, logging, os
from gensim import corpora
from collections import defaultdict
from pprint import pprint
import numpy as np
from gensim.models import Word2Vec
import time
t1 = time.time()

In [ ]:
sentences[0]

In [ ]:
texts = [[word for word in text_cleaner(line.strip().split('\t')[0])] for line in sentences]

In [ ]:
texts

In [ ]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
model = Word2Vec(
    sentences=texts,
    min_count=0,
    sample=0.001,
    seed=1,
    workers=8,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    epochs=100,
    vector_size=50
)

In [ ]:
vector = model.wv['unjustified']
print(vector)

In [ ]:
sims = model.wv.most_similar('responsible', topn=10)
for word, score in sims:
    print(f"{word}: {score:.4f}")

In [ ]:
result = model.wv.most_similar(positive=['healthcare', 'clinics'], negative=['death'], topn=1)
print(result) # Expected output: [('queen', 0.7118)]

# FastText

In [ ]:
stopwords = ['rt','amp','url','sir','day','title','shri','crore','time',"a", "about","above", "across", "after", "afterwards", "again", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
tstop = ['rt','amp','url']
import itertools

import re

def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b"," ",newString)
    newString = re.sub("[^a-zA-Z_@#]", " ", newString)
    long_words=[]
    for i in newString.split():
        if i not in tstop:
            long_words.append(i)
    return long_words

import emoji

def remove_link(text):
    regex = r'https?://[^\s<>)"‘’]+'
    match = re.sub(regex,' ', text)
    regex = r'urls?'
    match = re.sub(regex,' ', match)
    tweet=emoji.demojize(match)
    tweet = re.sub("[0-9:;,.()!?…]", " ", tweet)
    tweet = re.sub("[ ]+", " ", tweet)
    return tweet.strip()

In [ ]:
import nltk

In [ ]:
word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
embedding_size = 60
window_size = 10
min_word = 5
down_sampling = 1e-2

In [ ]:
ft_model = FastText(
    sentences=word_tokenized_corpus,
    vector_size=embedding_size,
    window=window_size,
    min_count=min_word,
    sample=down_sampling,
    sg=1,
    epochs=100
)

In [ ]:
ft_model.wv.most_similar(['responsible'], topn=10)

In [ ]:
result = ft_model.wv.most_similar(positive=['healthcare', 'clinics'], negative=['death'], topn=1)

print(f"Analogy result: {result[0][0]}")